# Observation Planning Part 2: Sensitivity Calculation

1. Where is my target?
2. When can I observe my target?
3. How do I know when I've found my target (make a finder chart)?
4. __How long do I need to observe?__
5. How will I calibrate my data?

In Observation Planning part 1, we covered parts 1,2,3 above.  Now we'll cover the 4th part, calculating observation times.

First, a brief review:  For each of the following questions, give a brief explanation of what tools you use to answer the first three questions:

(1) Where is my target?

Your answer here

(2) When can I observe my target?

Your answer here

(3) How do I know when I've found my target?


Your answer here

## How long do I need to observe my target?

We know approximately how long we _can_ observe each target, but how long do we _need_ per source?  To answer this question, we need to do some sensitivity calculations.


First, we determine how bright our target is.  You can only obtain this information for known sources, but generally, we can set a reasonable limit based off of physical considerations.  In other words, as long as we know what we're looking for, we can make a pretty good guess of how long we need to look.

Our observations will be performed in the visual, meaning we are interested in the source brightness in the R (red) and V (visual) filters.

We can use SIMBAD to look up the typical brightness of the target source in magnitudes.  You can go on the SIMBAD website to see how this works, but astroquery can also do it, so we're going to use astroquery here.  The astroquery SIMBAD module tells you what "fields" (data column names) are available to query on for each source.

In [66]:
from astroquery.simbad import Simbad

Recall from the previous workbook that we can use SIMBAD to retrieve information about stars, but the information included does not, by default, include the flux (or magnitude) of the source.

In [67]:
pcyg_simbad = Simbad.query_object('P Cygni')
pcyg_simbad

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,FLUX_R,FLUX_V,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,mag,mag,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,float32,float32,int32
* P Cyg,20 17 47.2020,+38 01 58.552,11,11,0.120,0.130,90,A,O,2007A&A...474..653V,4.28,4.82,1


We add the flux fields for the R and V filters, then we can see the two new columns added on the right:

In [68]:
Simbad.reset_votable_fields()
Simbad.add_votable_fields('flux(R)', 'flux(V)')
pcyg_simbad = Simbad.query_object('P Cygni')
pcyg_simbad

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,FLUX_R,FLUX_V,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,mag,mag,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,float32,float32,int32
* P Cyg,20 17 47.2020,+38 01 58.552,11,11,0.120,0.130,90,A,O,2007A&A...474..653V,4.28,4.82,1


In [69]:
pcyg_simbad['FLUX_R'].data

masked_array(data=[4.28000020980835],
             mask=[False],
       fill_value=1e+20,
            dtype=float32)

In [70]:
pcyg_simbad['FLUX_R','FLUX_V']

FLUX_R,FLUX_V
mag,mag
float32,float32
4.28,4.82


P Cygni is about 4.3 magnitudes in R.  What does that mean?



## Magnitudes


Magnitudes are a weird system with relative definitions.  The definition of magnitude is:

$$ m = -2.5 \log(F / F_0) $$

$m$ is the observed (apparent) magnitude, $F$ is the source flux (in units of Jansky, ergs/cm$^2$/angstrom, W/m$^2$/angstrom, or erg/cm$^{2}$/Hz, usually), and $F_0$ is the flux at magnitude zero. 

__$F_0$ is a definition!__

It was originally referenced to Vega, but more accurate observations have revealed that Vega is itself variable, so now the absolute standard is set by physical units.

So, we have to look up $F_0$ or determine it ourselves.

Note that each filter will have its own zero-point (also, terminology aside: the "zero point of a filter" and the "photometric zero point" are different things).  For example, for the visible $V$ band, we say:

$$ m_V = -2.5 \log(F_V / F_{0,V}) $$
or for red:
$$ m_R = -2.5 \log(F_R / F_{0,R}) $$

In the red filter, the zero-point is approximately 2500 Jy (which we just look up).  Therefore, we can compute the flux from P Cygni:

$$ m_{P Cyg} = 4.3 = -2.5 \log(F_{P Cyg} / 2500 \mathrm{Jy}) $$

Solve for $F_{P Cyg}$:

$$ F_{P Cyg} = 10^{(-4.3/2.5)} \times 2500 \mathrm{Jy} $$

In [71]:
from astropy import units as u

In [72]:
F_PCyg = 10**(-4.3/2.5) * 2500*u.Jy
F_PCyg

<Quantity 47.63651795 Jy>

This tells us the amount of energy received per area per unit frequency at the telescope.  Next, we need to compute the number of photons we'll collect.

1 Jansky is $10^{-23}$ ergs s$^{-1}$ cm$^{-2}$ Hz$^{-1}$ (https://en.wikipedia.org/wiki/Jansky)

In [73]:
(1*u.Jy).to(u.erg/u.s/u.cm**2/u.Hz)

<Quantity 1.e-23 erg / (cm2 Hz s)>

## Modeling the telescope

We need to describe our instrument now.  Our telescope is a 14" diameter telescope, so its area is (pi * (radius)$^2$).  An inch is 2.54 centimeters.  So, our collecting area is $A_{CTO 14"}=$

In [74]:
import numpy as np

In [75]:
# student answer here
A_CTO = (np.pi*(14/2 * u.imperial.inch)**2).to(u.cm**2)
A_CTO

<Quantity 993.14665903 cm2>

This should be 993.14666 cm$^2$

## Modeling the filter

To determine how many photons we will receive, we need to know which wavelengths of photons are hitting the detector, and which ones are energetic enough to produce electrons through the photoelectric effect.

We do this by putting a filter in front of the detector, which prevents longer- or shorter-wavelength (lower- or higher-energy) photons from getting through.

We will approximate that our red filter is $\sim$1000 angstroms wide (in reality, there is a [wavelength-dependent filter function we could use](http://voservices.net/filter/), but we will take the simple approach for now).  The width of a filter is relative to the wavelength, and the red filter is centered at about 6500 angstroms.

To convert a central wavelength from angstroms to Hertz, we use the definition $\nu  \lambda = c$, i.e., the frequency times the wavelength is the speed of light. 

For example, 100 microns ($\mu$m) is at 3 THz.  Astropy's `constants.c` is the speed of light:

In [76]:
from astropy import constants
wavelength = 100*u.um
frequency = constants.c  / wavelength
frequency

<Quantity 2997924.58 m / (s um)>

Note the units on that are something funky, $m \mu^{-1} s^{-1}$.  You can use astropy's `.to` to convert to a specific unit, or `.decompose` to reduce to a minimal expressible unit:

In [77]:
print("Decomposed frequency: ", frequency.decompose())
print("Frequency in THz: ", frequency.to(u.THz))

Decomposed frequency:  2997924580000.0 1 / s
Frequency in THz:  2.99792458 THz


So, returning to our red filter, for a wavelength $\lambda = 6500$ angstroms, solve the equation $\lambda \nu = c$ for $\nu$ in Hz or THz:

In [78]:
# student answer here
nu = (constants.c / (6500*u.AA)).to(u.Hz)
nu

<Quantity 4.61219166e+14 Hz>

Should be $4.6121917×10^{14}$ Hz

The filter width is $\Delta\lambda / \lambda$.  We have defined the width $\Delta \lambda = 1000$ angstroms, and the central wavelength is $\lambda=6500$ angstroms.  The relative width is the same in both units, i.e.:
$$ \frac{\Delta\lambda}{\lambda} = \frac{\Delta\nu}{\nu} $$

You solved for $\nu$ above, so now determine $\Delta \nu$ (again, in Hz or THz)

In [79]:
# student answer here
dlam = 1000*u.AA
lam = 6500*u.AA # note we can't use the variable "lambda" because it's a reserved word in python
dnu = dlam/lam * nu
dnu

<Quantity 7.09567948e+13 Hz>

Your answer should be $7.0956795×10^{13}$Hz

We can convert our flux in Janskys to $10^{-23}$ ergs s$^{-1}$ cm$^{-2}$ Hz$^{-1}$, then multiply by the bandwidth in Hz and the collecting area in cm$^{2}$ to obtain the energy we will receive on the detector per second:

$$ E_{rec} = F_{P Cyg} * A_{CTO 14"} * \Delta \nu $$

Compute $E_{rec}$ in units of erg $s^{-1}$ (`u.erg/u.s`):

In [80]:
# you should have these three variables defined already
dnu, F_PCyg, A_CTO

(<Quantity 7.09567948e+13 Hz>,
 <Quantity 47.63651795 Jy>,
 <Quantity 993.14665903 cm2>)

In [81]:
# student answer here (in erg/s)
E_rec = F_PCyg * A_CTO * dnu
E_rec.to(u.erg/u.s)

<Quantity 3.35696941e-05 erg / s>

Should be: $3.3569694×10^{−5}$ ergs / s

The CCD detector responds to individual photons, so we need to count how many photons are coming in.  The average wavelegnth of a photon is 6500 angstroms (approximately).  We can compute the energy per photon using

$$ E_{phot} = h \nu$$

where $h$ is [Planck's constant](https://en.wikipedia.org/wiki/Planck_constant).  Using the above information, compute the mean energy per photon.

(recall that frequency and wavelength are related as $c = \nu / \lambda$)

In [82]:
# student answer here (compute the answer in both eV and erg)
E_phot = (constants.h * nu)
E_phot.to(u.eV), E_phot.to(u.erg)

(<Quantity 1.90744921 eV>, <Quantity 3.05607055e-12 erg>)

You should obtain a number near 2 eV.

Given our energy received from the star $F_{E} =$ `E_rec` [erg/s] and the energy per photon $E_{phot}$ = `E_phot`[erg], how many photons are we receiving per second $F_{\gamma}$?

$F_{\gamma} [\mathrm{phot~s}^{-1}] = F_{E} / E_{phot}$

In [83]:
# student answer here (units (1/u.s))
phot_per_s = (E_rec / E_phot).to(u.s**-1)
phot_per_s

<Quantity 10984593.97222623 1 / s>

Answer should be: 10984594 s$^{-1}$

CCD detectors saturate at approximately $2^{16} = 65536$ counts (though ours saturate at closer to 30,000).  Is the photon count greater than or less than this number? 

student answer:
greater than

If the above is greater than the saturation count, how many pixels do we need to spread the photons over if we want the image to stay below the saturation limit in a 1 second exposure?

In [84]:
# student answer
npix = phot_per_s * 1*u.s / 2**16
npix

<Quantity 167.61160236>

The CCDs at CTO are ST-402 cameras.  They have 765x510 pixels over 6.9x4.3mm.

How big are the individual pixels, in microns? (should be about 9x9 microns)

In [85]:
# student answer
(6.9*u.mm / 765).to(u.um), (4.3*u.mm / 510).to(u.um)

(<Quantity 9.01960784 um>, <Quantity 8.43137255 um>)

By observing a pair of stars with known separation on the sky, we can determine what the _pixel scale_ of our CCD is.  

However, for the purpose of this exercise, we will simply assume that the pixel scale is $\approx0.5"$ per pixel.

Over what radius circle does our star need to be projected?

We calculate this in two steps:  First, we know that the _area_ we need to project onto must be ~168 pixels.  What _radius_ circle has an area of 168 pixels?

In [86]:
# student answer
radius_pixels = (168/np.pi)**0.5
radius_pixels

7.312732791431452

Should be 7.3 pixels

What does that size in pixels correspond to in arcseconds?

In [87]:
# the "pixel scale" is the number of arcseconds across a pixel,
# so the units are "arcseconds per pixel",
# but we leave the "per pixel" part implicit (otherwise the code gets confusing)
pixel_scale = 0.5*u.arcsec

In [88]:
radius_pixels * pixel_scale

<Quantity 3.6563664 arcsec>

Is the radius we need to smear the source over, i.e., the radius in arcseconds we just calculated above, bigger or smaller than the expected source size (the "point spread function") given the typical seeing conditions at CTO ($\sim2"$)?

your answer here

Given that answer, can we observe this target without saturating the CCD?  Justify your answer.

your answer here

## Integration Times

In the above, we were simply determining whether the source would be above or below the saturation level of our CCDs.  Let's now determine how long we need to integrate to achieve a target signal to noise ratio.

We decide our integration time based on a target signal-to-noise ratio (SNR).  If our target SNR is 100, how long do we need to integrate?

To a first approximation, we can assume that the signal-to-noise is limited by counting statistics.

The signal $S$ rises as the count rate $\dot{p}$ times the time,  $S = \dot{p} t$.

The noise $N$ is, in the Poisson noise limit, the square root of the signal (in photons), i.e., $N = \sqrt{S} = \sqrt{\dot{p}t}$.  The SNR is therefore 
$$S/N = \frac{\dot{p} t}{\sqrt{\dot{p} t}} = \sqrt{\dot{p} t}$$

## Warning / Note

When we say that the uncertainty is the square root of the signal when counting photons, we have a bit of a units problem!

$$ \sigma = \sqrt{\lambda}$$

The number of photons counted has units: number of photons.  The _uncertainty_ on that number has the same units, despite the fact that, naively, you would look at the above equation and say that the uncertainty has units of $\sqrt{counts}$.

You can't rely on the astropy unit system (or any automated unit system) to handle this equivalency; you need to take care to establish the correct units when dealing with uncertainties on counts.

We calculated the count rate above ($\dot{p}$ = `phot_per_s`), so what integration time $t$ do we need for $SNR=100$ ?

(solve the above equation, $SNR = \sqrt{\dot{p} t}$, for $t$)

In [89]:
# student answer
integration_time = (phot_per_s)**-1 * 100**2
integration_time

<Quantity 0.00091037 s>

You should note that that is a pretty short time! (about 0.9 milliseconds)

What if we account for read noise?  Let's assume that we are averaging over a 2 arcsecond Gaussian PSF.  First, how many pixels is that?

The area of a Gaussian isn't exactly that of a circle.  $A_{2D \mathrm{Gaussian}} = 2 \pi \sigma^2$

In [90]:
psf_area = 2 * np.pi * (2*u.arcsec)**2
psf_area

<Quantity 25.13274123 arcsec2>

In [91]:
psf_area_pixels = psf_area * pixel_scale**-2
psf_area_pixels

<Quantity 100.53096491>

Given that our readnoise is 10 counts per pixel, and we're interested in the _sum_ over these pixels, what is the expected contribution to the noise on the sum from these pixels?

Remember the propagation of error formula for summation:
$$sum = \Sigma x_i$$
$$\sigma_{sum}^2 = \Sigma_i \sigma_i^2$$

In [92]:
# student answer
readnoise_per_pix = 10*u.adu
readnoise_sum = psf_area_pixels**0.5 * readnoise_per_pix
readnoise_sum

<Quantity 100.26513099 adu>

By coincidence, that's about 100.265 ADU

Given our calculation of the SNR and count noise above, what's our expected _total_ noise in the measurement over the PSF? (recall: the signal will be $t \times \dot{p}$, and we have calculated both of those quantities above)

In [93]:
# student answer (note: this value should be in the same units as the read noise above - you might have to add those units!)
photon_noise = (integration_time * phot_per_s)**0.5*u.adu
photon_noise

<Quantity 100. adu>

In [94]:
# student answer
total_noise = (photon_noise**2 + readnoise_sum**2)**0.5
total_noise

<Quantity 141.60895625 adu>

Should be about sqrt(2) * 100

Given that our noise is higher than we originally thought, we will not achieve our target signal to noise ratio if we use the `integration_time` above!  What will we achieve instead?

In [95]:
# student answer
signal = integration_time * phot_per_s * u.adu
snr = signal / total_noise
snr

<Quantity 70.61700237>

We can write down a different equation for the signal-to-noise ratio now that we know how much readnoise to expect for a stellar observation.

We have just computed that our total noise is $N = {\sqrt{\dot{p} t + \sigma_{rn}^2}}$, so our SNR is:

$$S/N = \frac{\dot{p} t}{\sqrt{\dot{p} t + \sigma_{rn}^2}} \approx \frac{\dot{p} t}{\sqrt{\dot{p} t + 100}} $$

This equation is no longer trivially solvable, but it can be rearranged nicely enough into a quadratic equation:
$$SNR^2 \left( \dot{p} t + 100^2\right) = \left(\dot{p}t\right)^2$$ 


if we substitute $S = \dot{p} t$, it looks a little simpler
$$ S^2 - S \cdot SNR^2  - 100^2 SNR^2 = 0$$
then, we can use the quadratic formula to solve for the needed signal:
$$S = SNR^2 \pm \frac{\sqrt{SNR^4 + 4 \cdot 100^2 \cdot SNR^2}}{2}$$

In [96]:
SNR = 100
target_signal = SNR**2 + (SNR**4 + 4*readnoise_sum.value**2*SNR**2)**0.5 / 2
target_signal

21204.060197753017

In [97]:
integration_time_including_readnoise = target_signal / phot_per_s
integration_time_including_readnoise

<Quantity 0.00193035 s>

In [98]:
# how much longer is that?
integration_time_including_readnoise / integration_time

<Quantity 2.12040602>

So, accounting for readnoise, to get to the same SNR, we need to integrate for about twice as long (2.12x).

## What would we need to be even more accurate in our prediction?

Note that all of these calculations make a few assumptions that are generally not correct:

1. We assumed the V-filter has a uniform transmission of 1 across its band.  In reality, the V-filter transmits closer to ~75% of the total band, so the total photon rate (in any given filter) is less than what we calculated above.

2. We assumed that the CCD has perfect _quantum efficiency_.  The best CCDs have close to unity (one) efficiency, but not exactly.  Ours are probably in the 60-80% range.

3. We assumed no flux losses to the atmosphere (absorption or scattering) or the telescope optics.  We know from class that the atmosphere absorbs anywhere from 10-30% of the incident flux if we point straight up on a clear night.

Together these effects amount to a substantial loss of light.  Let's estimate how much:

In [99]:
filter_efficiency = 0.75
quantum_efficiency = 0.7
atmosphere_loss = 0.2
received_fraction = filter_efficiency * quantum_efficiency * (1-atmosphere_loss)
received_fraction

0.41999999999999993

We're getting less than 50% of the light recorded on our CCD.  That means we're liable to underestimate the brightness of our targets by a lot!  Next time, we'll go over how we account for that loss by _calibrating_.

## Do it yourself for a more reasonable target.

We noted above that P Cygni is really hard to observe, since it will saturate our detector.

Now, let's repeat the above exercises for a different target: `TYC 2788-108-1`

Answer the following questions (using the code you learned above):

1.  How bright is TYC 2788-108-1 in the [Johnson V-band](http://voservices.net/filter/filterdtls.aspx?filterid=60)?  Give your answer in magnitudes and in Janskys, assuming the zero-magnitude flux is 3500 Jy (as per [the VO filter service](http://svo2.cab.inta-csic.es/theory/fps/index.php?id=Generic/Bessell.V&&mode=browse&gname=Generic&gname2=Bessell#filter))

In [100]:
Simbad.query_object('TYC 2788-108-1')

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,FLUX_R,FLUX_V,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,mag,mag,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,float32,float32,int32
TYC 2788-108-1,00 39 48.0058,+40 54 40.676,14,14,0.035,0.048,90,A,O,2018yCat.1345....0G,--,10.65,1


In [101]:
fluxv_tyc = 10.65
snu_tyc = 3500 * (10**(-10.65/2.5)) *u.Jy
snu_tyc

<Quantity 0.19233931 Jy>

2. Using the same telescope as above, what is our expected received flux in ergs/second and in photons per second?  Assume the V-filter band width is 870 Angstroms and the band center is 5504 Angstroms.

In [102]:
v_filt_freq = (5504*u.AA).to(u.Hz, u.spectral())
tyc_ergs_per_s = snu_tyc * A_CTO * v_filt_freq * (870/5504)
tyc_ergs_per_s.to(u.erg/u.s)

<Quantity 1.64461609e-07 erg / s>

In [103]:
tyc_phot_per_s = (tyc_ergs_per_s / (constants.h * v_filt_freq)).decompose()
tyc_phot_per_s

<Quantity 45568.65689848 1 / s>

3. How many photons per second per pixel will the star produce if the point spread function, the area over which the star's light is spread, is the same as above ($\sigma_{Gaussian} = 2"$)?  How long can we integrate without saturating (assuming we saturate at 30,000 counts)?

In [104]:
saturation = 30000

In [105]:
tyc_phot_per_s_per_pix = tyc_phot_per_s / psf_area_pixels
tyc_phot_per_s_per_pix

<Quantity 453.27981222 1 / s>

In [106]:
time_to_saturation = saturation / tyc_phot_per_s_per_pix
time_to_saturation.to(u.s)

<Quantity 66.18428439 s>

4. Again following the same assumptions (the same spread among pixels, same readnoise), how long do we need to integrate to get to a signal-to-noise ratio of 100?  Account for both photon statistics and readnoise

In [107]:
SNR = 100
target_signal = SNR**2 + (SNR**4 + 4*readnoise_sum.value*SNR**2)**0.5 / 2
target_signal

15099.279489285953

In [108]:
integration_time_including_readnoise = target_signal / tyc_phot_per_s
integration_time_including_readnoise

<Quantity 0.3313523 s>